# ReAct Agent
`ReAct`는 **"Reasoning and Acting"** 의 줄임말로, LLM(거대 언어 모델)이 단순히 답변을 생성하는 것을 넘어,  
목표를 달성하기 위해 어떤 **도구(Tool)** 를 사용해야 할지 **추론(Thought)** 하고, 그에 따라 **행동(Action)** 을  
 수행하는 과정을 반복하도록 만드는 프레임워크

### React의 작동 방식
- `질문(Question)`: 사용자의 초기 질문을 받습니다.
- `생각(Thought)`: LLM은 질문을 해결하기 위해 어떤 도구를, 어떤 입력값으로 사용해야 할지 계획을 세우고 추론합니다.
- `행동(Action)`: 추론에 따라 사용할 도구(Tool)와 그 도구에 전달할 입력(Action Input)을 결정합니다.
- `관찰(Observation)`: 행동(Action)을 통해 실행된 도구의 결과를 받습니다.
- `반복`: LLM은 관찰(Observation)된 결과를 바탕으로 최종 답변을 할 수 있는지 판단합니다.  
    만약 작업이 끝나지 않았다면, 다시 생각(Thought) 단계로 돌아가 다음 행동을 계획합니다. 이 과정은 최종 답변(Final Answer)에 도달할 때까지 반복됩니다.

In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

OPENAI_API_KEY가 설정되어 있나요?: sk-proj-WS...


#### RAG 구성

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool


loader = PyPDFLoader("../data/[AI.GOV_해외동향]_2025-1호.pdf")
# 문서 읽어오기
docs = loader.load()

# 문서 분할 (Chunk 단위로 분할할)
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "],  # 문단 → 줄 → 문장 → 단어 순
    chunk_size=1000,    
    chunk_overlap=100
)

chunks = recursive_splitter.split_documents(docs)

# 임베딩 모델 준비
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

# 벡터 스토어 생성
db = FAISS.from_documents(documents=chunks, embedding=embeddings)

# Retriever 생성
# 2. MMR Search  
mmr_retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,  # 검색 문서 수를 3개로 설정
        "lambda_mult": 0.8,  # 다양성 중시
    }
)

# Retriever를 Agent가 사용할 수 있는 도구로 변환
retriever_tool = create_retriever_tool(
    retriever=mmr_retriever,
    name="pdf_search",
    description="국가별 AI 동향에 대해 PDF 문서 내에서 검색해 반환합니다."
)

Tavily API를 사용해 검색 도구 구현

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 검색 도구 생성
web_search_tool = TavilySearchResults(
    max_results=3,
    include_answer=True,
    include_raw_content=False,
    search_depth="advanced"
)

C:\Users\jtw57\AppData\Local\Temp\ipykernel_24352\408039398.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(


In [4]:
tools = [web_search_tool,retriever_tool]

#### create_react_agent
- **ReAct** 프레임워크를 사용하여 스스로 추론하고 행동하는 에이전트(Agent)를 생성하는 함수


**주요 파라미터**  

`llm (필수)` : 에이전트의 "뇌" 역할을 하는 언어 모델입니다.  
- 주로 Chat 모델(예: ChatOpenAI)을 사용하며, 추론(Thought)과 행동(Action)을 결정하는 핵심 요소입니다.

`tools (필수)`: 에이전트가 사용할 수 있는 도구들의 리스트입니다.  
- 각 도구는 특정 기능을 수행하는 함수로 정의됩니다. (예: 웹 검색 도구, 데이터베이스 조회 도구, 계산기 등)  
- 에이전트는 이 도구들의 설명(description)을 보고 어떤 상황에 어떤 도구를 사용해야 할지 판단합니다.

`prompt (필수)` : 에이전트가 ReAct 프레임워크에 따라 작동하도록 지시하는 템플릿입니다.
- 이 프롬프트에는 반드시 3개의 입력 변수가 포함되어야 합니다.
    - tools: 사용 가능한 도구들의 이름과 설명이 들어갑니다.
    - tool_names: 사용 가능한 도구들의 이름 목록이 들어갑니다.
    - agent_scratchpad: 이전 단계의 생각(Thought), 행동(Action), 관찰(Observation) 기록이 누적되는 공간입니다.  
    이를 통해 에이전트는 지금까지의 작업 내용을 기억하고 다음 단계를 계획할 수 있습니다.


주의할점
- LangChain react_agent의 내장파서는 정규식으로 영어키워드를 사용한다.
**("Question", "Thought", "Action", "Action Input", "Observation", "Final Answer")**
- 프롬프트에 한글버전으로 작성하게되면 ReAct 프롬프트에 있는 이 키워드들을 인식하지못해 정상적으로 동작하지 않음.


1. 프롬프트 생성

In [5]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate

# ReAct 프롬프트의 구조를 정의하는 템플릿 문자열을 작성합니다.
react_prompt_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

rule:
- response must be in Korean

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

# 위 템플릿 문자열로부터 프롬프트 객체를 생성합니다.
prompt = PromptTemplate.from_template(react_prompt_template)

# LLM 정의
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

2. React Agent 생성

In [6]:
from langchain.agents import create_react_agent

agent = create_react_agent(llm, tools, prompt)

3. AgentExecutor 생성

In [7]:
from langchain.agents import AgentExecutor

# Agent Executor 생성
agetnt_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # 실행 과정을 자세히 보여줍니다
    handle_parsing_errors=True,  # 파싱 에러를 자동으로 처리합니다
    max_iterations=5 # 최대 5번만 "생각/행동"을 반복합니다
)

In [8]:
result = agetnt_executor.invoke({"input": "중국의 AI 동향에 대해서 문서내에서 검색해 알려줘"})
print(result)



> Entering new AgentExecutor chain...
중국의 AI 동향에 대한 정보를 찾기 위해 관련 PDF 문서에서 검색해야 할 것 같다.  
Action: pdf_search  
Action Input: '중국 AI 동향'  1
해외 에이전트 기술 및 서비스 개발 동향AI 
을 보완하는 기술 현황LLM RAG 2.0 
정부의 적극적 활용 정책과 반향 미국 영국 사례AI : ·
주요국 인프라 투자 경쟁 가속화AI 
최신 보고서 의 미래 예견적 거버넌스 전략[OECD ] AI :

중국 스타트업 의 등 에이전트 서비스 잇따라 출시Butterfly Effect ‘Manus AI’ AI 
   - 다만 아직 주요 기능은 실험적이거나 초기 베타 버전으로 제공되고 있음, ( ) 
1) IBM. (2024, January 3) What are AI agents? https://www.ibm.com/think/topics/ai-agents
2) Gartner. (2024. October 21) Top 10 Strategic Technology Trends for 2025.
https://www.gartner.com/en/articles/top-technology-trends-2025
3) Fortune. (2025, January 7) Nvidia’s Jensen Huang says AI agents are ‘a multi-trillion-dollar opportunity’ and 
‘the age of AI Agentics is here’. https://finance.yahoo.com/news/nvidia-jensen-huang-says-ai-044815659.html

해외동향 호AI.GOV 2025-1 13
   - 미국 조달청 은 직원들의 생산성 향상이라는 명목으로 챗봇 을 시범 (GSA) (GSA Chat)
도입하였으나 실질적으로는 인력 감축을 위한 전략의 일환이라는 분석이 존재, 12)
   - 는 교육부를 비롯한 여러 연방기관에서 를 활용

In [9]:
response = agetnt_executor.invoke(
    {"input": 
    "중국의 AI 동향에 문서내에서 검색해 알려줘 만약 내용이 부족하다면 관련된 추가적인 정보를 웹검색을 통해 정리해줘"})
print(response["output"])



> Entering new AgentExecutor chain...
중국의 AI 동향에 대한 문서를 찾기 위해 PDF 내에서 검색을 진행하겠습니다.  
Action: pdf_search  
Action Input: '중국 AI 동향'  1
해외 에이전트 기술 및 서비스 개발 동향AI 
을 보완하는 기술 현황LLM RAG 2.0 
정부의 적극적 활용 정책과 반향 미국 영국 사례AI : ·
주요국 인프라 투자 경쟁 가속화AI 
최신 보고서 의 미래 예견적 거버넌스 전략[OECD ] AI :

중국 스타트업 의 등 에이전트 서비스 잇따라 출시Butterfly Effect ‘Manus AI’ AI 
   - 다만 아직 주요 기능은 실험적이거나 초기 베타 버전으로 제공되고 있음, ( ) 
1) IBM. (2024, January 3) What are AI agents? https://www.ibm.com/think/topics/ai-agents
2) Gartner. (2024. October 21) Top 10 Strategic Technology Trends for 2025.
https://www.gartner.com/en/articles/top-technology-trends-2025
3) Fortune. (2025, January 7) Nvidia’s Jensen Huang says AI agents are ‘a multi-trillion-dollar opportunity’ and 
‘the age of AI Agentics is here’. https://finance.yahoo.com/news/nvidia-jensen-huang-says-ai-044815659.html

해외동향 호AI.GOV 2025-1 13
   - 미국 조달청 은 직원들의 생산성 향상이라는 명목으로 챗봇 을 시범 (GSA) (GSA Chat)
도입하였으나 실질적으로는 인력 감축을 위한 전략의 일환이라는 분석이 존재, 12)
   - 는 교육부를 비롯한 여러 연방기관에서 를 활용해 정부